### 0.대전광역시_대전교통정보 open API
#### - 변수설명

1. congestion : 혼잡코드   
  - 0:정보없음 1:소통원활 2:지체 3:정체
2. linkCount: 링크 개수
3. linkID: 링크식별번호
4. linkLength: 링크길이
5. linkSqc:	링크정렬순번
6. speed: 통행속도(Km/h)
7. roadName: 도로명칭	
8. startNodeId: 시작노드ID
9. 	startNodeName: 시작노드명
10. endNodeID: 끝노드ID
11. endNodeName: 끝노드명	
12. travelT: 통행시간(초)	
8. udType: 방향정보   
  - 0:정보없음, 1:상행, 2:하행

### 1. 필요 라이브러리

In [1]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

### 2. Open API 데이터 불러오기

In [17]:
# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://openapitraffic.daejeon.go.kr/traffic/rest/getTrafficInfoAll.do'
My_API_Key = unquote('DZmq1Zbk1ohdHMUAi%2BysDvzhfimwdCa5S7Zkrstme%2FxcK33DZYmjRugvRb9JtpMh9Ef%2F1b%2FyYrm1ftXE891wIw%3D%3D')    


In [18]:
# My_API_Key = unquote('Agq7hySmyMi1FFU9kYibP%2BEnxYepQ%2FB6Dn%2Bw9lsYKVSCDjTwIdvpjmuhJrtyQrhipg3F3a4jbSq%2FLxHi%2FdUIoQ%3D%3D')    # 사용자 인증키
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : My_API_Key,    # 필수 항목 1 : ServiceKey
        quote_plus('pageNo') : 1,          # 필수 항목 2 : pageNo
        quote_plus('numOfRows') : 15000         # 픽수 항목 3 : 	numOfRows
     }
)

response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
# xmlobj    # 디버깅용.

In [19]:
rows = xmlobj.findAll('TRAFFIC')

In [20]:
rows

[<TRAFFIC>
 <congestion>1</congestion>
 <linkCount>14444</linkCount>
 <linkID>1830000100</linkID>
 <linkLength>145.293874194</linkLength>
 <linkSqc>1</linkSqc>
 <speed>30</speed>
 <roadName>일반국도17호선</roadName>
 <startNodeId>1830000100</startNodeId>
 <startNodeName>추부터널</startNodeName>
 <endNodeID>2920011700</endNodeID>
 <endNodeName>추부터널</endNodeName>
 <travelT>5</travelT>
 <udType>2</udType>
 </TRAFFIC>,
 <TRAFFIC>
 <congestion>1</congestion>
 <linkCount>14444</linkCount>
 <linkID>1830000200</linkID>
 <linkLength>145.940446066</linkLength>
 <linkSqc>2</linkSqc>
 <speed>30</speed>
 <roadName>태봉길</roadName>
 <startNodeId>2920011700</startNodeId>
 <startNodeName>추부터널</startNodeName>
 <endNodeID>1830000100</endNodeID>
 <endNodeName>추부터널</endNodeName>
 <travelT>5</travelT>
 <udType>1</udType>
 </TRAFFIC>,
 <TRAFFIC>
 <congestion>1</congestion>
 <linkCount>14444</linkCount>
 <linkID>1830000301</linkID>
 <linkLength>210.42007237</linkLength>
 <linkSqc>3</linkSqc>
 <speed>30</speed>
 <roadNam

In [21]:
len(rows)

14444

In [22]:
#한개 행 모든컬럼값 리스티에 담기
columns = rows[0].find_all()
columns

[<congestion>1</congestion>,
 <linkCount>14444</linkCount>,
 <linkID>1830000100</linkID>,
 <linkLength>145.293874194</linkLength>,
 <linkSqc>1</linkSqc>,
 <speed>30</speed>,
 <roadName>일반국도17호선</roadName>,
 <startNodeId>1830000100</startNodeId>,
 <startNodeName>추부터널</startNodeName>,
 <endNodeID>2920011700</endNodeID>,
 <endNodeName>추부터널</endNodeName>,
 <travelT>5</travelT>,
 <udType>2</udType>]

In [23]:
rowList = []
nameList = []
columnList = []

columnsLen = len(columns)
for j in range(0, columnsLen):
    eachColumn = columns[j].text
    columnList.append(eachColumn)
rowList.append(columnList)

In [24]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)

result = pd.DataFrame(rowList, columns=nameList)

In [25]:
result

,congestion,linkCount,linkID,linkLength,linkSqc,speed,roadName,startNodeId,startNodeName,endNodeID,endNodeName,travelT,udType
0,1,14444,1830000100,145.293874194,1,30,일반국도17호선,1830000100,추부터널,2920011700,추부터널,5,2
1,1,14444,1830000200,145.940446066,2,30,태봉길,2920011700,추부터널,1830000100,추부터널,5,1
2,1,14444,1830000301,210.42007237,3,30,태봉길,1830000100,추부터널,1830000101,하소동,7,1
3,2,14444,1830000302,10982.3911923,4,13,일반국도17호선,1830000101,하소동,1830001805,구도동,845,1
4,1,14444,1830000303,382.712429313,5,30,산내길,1830001805,구도동,1830001804,석천들주공아파트,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14439,1,14444,3080049702,16.4895,14435,30,지방도68호선,3080022502,-,3080022501,-,1,9
14440,1,14444,3080049703,212.164,14436,30,지방도68호선,3080022600,-,3080022502,-,7,9
14441,1,14444,3080049801,192.215,14437,30,지방도68호선,3080022500,-,3080022501,-,6,9
14442,1,14444,3080049802,16.4894,14438,30,지방도68호선,3080022501,-,3080022502,-,1,9


In [100]:
result.to_excel('대전교통정보.xlsx',index=False)